# Stochastic Galerkin
## Single parameter

### Linear system $Au = f$

In [ ]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse
import sys
import time

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

n = 125 #number of grid points in each spatial direction
h = 1/(n+1) #grid spacing
K = 1 #order of PCE
p = int((np.math.factorial(K+1)/np.math.factorial(K)) - 1) #p+1=((k+r)!/k!r!)+1, here r=1

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Inner products
def ip(k,i):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)*legendre(i,eps)
    return quad(I,-1,1)[0]

def ip2(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)
    return quad(I,-1,1)[0]

def ip3(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)**2
    return quad(I,-1,1)[0]

#Define central differencing operator matrix using five point stencil 
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
L = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n))

#Define inner product matrix
P = np.zeros([p+1,p+1]) 
for i in range(0,p+1):
    for j in range(0,p+1):
        P[i][j] = ip(i,j)
    
#Define F and vectorise
F1 = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y) 
F1 = np.reshape(F1,-1)

F2 = 34 * np.pi**2 * np.sin(3*np.pi*X) * np.sin(5*np.pi*Y)
F2 = np.reshape(F2, -1)

#Don't need to loop through all entries as most are zero
f = np.zeros([p+1,n**2]) 
f[0] = (F1*ip2(0))+(F2*ip3(0))
f[1] = (F1*ip2(1))+(F2*ip3(1))
if K > 1:
    f[2] = (F1*ip2(2))+(F2*ip3(2))
f = np.reshape(f,-1)

#Compute matrix D and covert to sparse format
D = kron(P,L)
D = sparse.csr_matrix(D)

start_time = time.time()

#Solve system and reshape appropriately
U = cg(D,f,tol=1e-9)[0]

total_time = time.time() - start_time

U = np.reshape(U,(p+1,n**2))
U = U.transpose()
mean_u = U[:,0]
mean_u = np.reshape(mean_u,(n,n))

#Define x and y as arrays between -1 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

U_expected = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

err_inf, err_sq = compute_err(mean_u, U_expected)
print('Err_inf:', err_inf, '\nErr sq:', err_sq, '\nTime taken:', total_time)

### Matrix form: $LXP = F$

In [ ]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.linalg import schur
from scipy.special import legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse
import sys
import time

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

n = 125 #number of grid points in each spatial direction
h = 1/(n+1) #grid spacing
K = 1 #order of PCE
p = int((np.math.factorial(K+1)/np.math.factorial(K)) - 1) #p+1=(k+r)!/k!

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Inner products
def ip(k,i):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)*legendre(i,eps)
    return quad(I,-1,1)[0]

def ip2(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)
    return quad(I,-1,1)[0]

def ip3(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)**2
    return quad(I,-1,1)[0]

#Define central differencing operator matrix using five point stencil 
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
L = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n))

#Inner product matrix
P = np.zeros([p+1,p+1]) 
for i in range(0,p+1):
    for j in range(0,p+1):
        P[i][j] = ip(i,j)
        
#Define F 
F1 = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y) 
F1 = np.reshape(F1,-1)

F2 = 34 * np.pi**2 * np.sin(3*np.pi*X) * np.sin(5*np.pi*Y)
F2 = np.reshape(F2, -1)

f = np.zeros([p+1,n**2]) 
f[0] = (F1*ip2(0))+(F2*ip3(0))
f[1] = (F1*ip2(1))+(F2*ip3(1))
if K > 1:
    f[2] = (F1*ip2(2))+(F2*ip3(2))

F = f.transpose()

start_time = time.time()

X = spsolve(L,np.matmul(F,np.linalg.inv(P))) #RHS is a matrix: FP^{-1}

total_time = time.time() - start_time

X = X[:,0]
mean_u = np.reshape(X,(n,n))

#Define x and y as arrays between -1 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

U_expected = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

err_inf, err_sq = compute_err(mean_u, U_expected)
print('Err inf', err_inf, '\nErr sq:', err_sq, '\nTotal time:', total_time)

### Matrix Form: $2LXG_0  + LXG_1 = F$

In [ ]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg, eigs
from scipy.sparse.linalg import inv as spinv
from scipy.linalg import schur, solve_sylvester
from scipy.special import legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse
import sys
import time

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

n = 250 #number of grid points in each spatial direction
h = 1/(n+1) #grid spacing
K = 1 #order of PCE
p = int((np.math.factorial(K+1)/np.math.factorial(K)) - 1) #p+1=(k+r)!/k!

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

def ip(k,i):
    I = lambda eps: (1/2)*legendre(k,eps)*eps*legendre(i,eps)
    return quad(I,-1,1)[0]

def ip2(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)
    return quad(I,-1,1)[0]

def ip3(k):
    I = lambda eps: (1/2)*legendre(k,eps)*(eps+2)**2
    return quad(I,-1,1)[0]

def ipg(k,i):
    I = lambda eps: (1/2)*legendre(k,eps)*legendre(i,eps)
    return quad(I,-1,1)[0]

#Define central differencing operator matrix using five point stencil 
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
L = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n))

G0 = np.zeros([p+1,p+1])
G1 = np.zeros([p+1,p+1])
for i in range(0,p+1):
    for j in range(0,p+1):
        G0[i][j] = ipg(i,j)
        G1[i][j] = ip(i,j)
                
#Define F 
F1 = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y) 
F1 = np.reshape(F1,-1)

F2 = 34 * np.pi**2 * np.sin(3*np.pi*X) * np.sin(5*np.pi*Y)
F2 = np.reshape(F2, -1)

f = np.zeros([p+1,n**2]) #Initalise f and set first row = e and reshape

f[0] = (F1*ip2(0))+(F2*ip3(0))
f[1] = (F1*ip2(1))+(F2*ip3(1))
if K > 1:
    f[2] = (F1*ip2(2))+(F2*ip3(2))

#Sparse
F = f.transpose()
G0_inv = np.linalg.inv(G0)

start_time = time.time()

#Setup LHS and RHS and solve for C
LHS = sparse.csc_matrix(kron(L,np.eye(p+1)))
RHS = np.matmul(F,G0_inv)
RHS = np.reshape(RHS,-1)
C = cg(LHS,RHS,tol=1e-9)[0]
C = np.reshape(C,(n**2,p+1))

#Ssolve for X using sim trans method
A = diags([[2]], [0], shape=(n**2, n**2))
B = sparse.csc_matrix(G1) @ sparse.csc_matrix(G0_inv)

vals, Q = np.linalg.eig(B.toarray())

C_hat = C @ Q

X_hat = np.zeros([n**2,p+1])

for i in range(0,p+1):
    X_hat[:,i] = C_hat[:,i] / (2+vals[i])

X = np.matmul(X_hat,np.linalg.inv(Q))

total_time = time.time() - start_time

X = X[:,0]
mean_u = np.reshape(X,(n,n))

#Define x and y as arrays between -1 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) + 2*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y)

err_inf, err_sq = compute_err(mean_u, U_exact)
print('Err inf', err_inf, '\nErr sq:', err_sq, '\nTotal time:', total_time)


## Multiple parameters 
### Linear System $Au = f$

In [ ]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse
import sys
import time

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

n = 125 #number of grid points in each spatial direction
h = 1/(n+1) #grid spacing
N = 2 #defines number of random variables = N^2
K = 1 #order of PCE
p = int((np.math.factorial(K+N**2)/(np.math.factorial(K)*np.math.factorial(N**2))))

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

def ip(k,i):
    I = lambda eps: (1/2**(N**4))*legendre(k,eps)*(eps)*legendre(i,eps)
    return quad(I,-1,1)[0]

def ip2(k):
    I = lambda eps: (1/2**(N**4))*legendre(k,eps)*(eps)
    return quad(I,-1,1)[0]


#Define central differencing operator matrix using five point stencil 
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
L = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n))

P = np.zeros([p+1,p+1]) 
for i in range(0,p+1):
    for j in range(0,p+1):
        P[i][j] = ip(i,j)
        
summation = np.zeros([n,n])
D = sparse.lil_matrix((n**2,n**2))
summation2 = np.zeros([n,n])
for i in range(0,n):
    for j in range(0,n):
        sum1 = 0
        sum2 = 0
        for a in range(0,N):
            for b in range(0,N):
                sum1 += 2**(-((a+1)+(b+1)))*np.sin((a+1)*np.pi*X[i][j])*np.sin((b+1)*np.pi*Y[i][j])
                sum2 += 2**(-((a+1)+(b+1)))*((a+1)**2+(b+1)**2)*np.sin((a+1)*np.pi*X[i][j])*np.sin((b+1)*np.pi*Y[i][j])
        D[i*n+j,i*n+j] = sum1
        summation[i][j] = sum1
        summation2[i][j] = sum2
        
                    
#Define F 
F1 = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y) * summation
F1 = np.reshape(F1,-1)

F2 = np.pi**2 * summation2
F2 = np.reshape(F2, -1)

f = np.zeros([p+1,n**2])

f[1] = (F1*ip2(1))+(F2*ip2(1))*(np.reshape(summation,-1)*ip2(1))
f = np.reshape(f,-1)

#LHS = sparse.csr_matrix(kron(P,D)@kron(np.eye(p+1),L))
LHS = sparse.csr_matrix(kron(P @ np.eye(p+1), D @ L))

start_time = time.time()

#Solve system and reshape appropriately
U = spsolve(LHS,f)

solve_time = time.time() - start_time

U = np.reshape(U,(p+1,n**2))
U = U.transpose()
mean_u = U[:,0]
mean_u = np.reshape(mean_u,(n,n))
    
#Define x and y as arrays between -1 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) 
err_inf, err_sq = compute_err(mean_u, U_exact)
print('Total time:', solve_time, '\nErr_inf:', err_inf, '\nErr sq:', err_sq)


### Matrix Equation DLXP = F

In [ ]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import legendre
from scipy.integrate import quad
from sympy import *
from scipy import sparse
import sys
import time

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

n = 125 #number of grid points in each spatial direction
h = 1/(n+1) #grid spacing
N = 2 #defines number of random variables = N^2
K = 1 #order of PCE
p = int((np.math.factorial(K+N**2)/(np.math.factorial(K)*np.math.factorial(N**2))))


#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

def ip(k,i):
    I = lambda eps: (1/2**(N**4))*legendre(k,eps)*(eps)*legendre(i,eps)
    return quad(I,-1,1)[0]

def ip2(k):
    I = lambda eps: (1/2**(N**4))*legendre(k,eps)*(eps)
    return quad(I,-1,1)[0]


#Define central differencing operator matrix using five point stencil 
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
L = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n))

P = np.zeros([p+1,p+1]) 
for i in range(0,p+1):
    for j in range(0,p+1):
        P[i][j] = ip(i,j)
        
summation = np.zeros([n,n])
D = sparse.lil_matrix((n**2,n**2))
summation2 = np.zeros([n,n])
for i in range(0,n):
    for j in range(0,n):
        sum1 = 0
        sum2 = 0
        for a in range(0,N):
            for b in range(0,N):
                sum1 += 2**(-((a+1)+(b+1)))*np.sin((a+1)*np.pi*X[i][j])*np.sin((b+1)*np.pi*Y[i][j])
                sum2 += 2**(-((a+1)+(b+1)))*((a+1)**2+(b+1)**2)*np.sin((a+1)*np.pi*X[i][j])*np.sin((b+1)*np.pi*Y[i][j])
        D[i*n+j,i*n+j] = sum1
        summation[i][j] = sum1
        summation2[i][j] = sum2
        
                    
#Define F 
F1 = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y) * summation
F1 = np.reshape(F1,-1)

F2 = np.pi**2 * summation2
F2 = np.reshape(F2, -1)

f = np.zeros([p+1,n**2])

for i in range(0,p+1):
    f[i] = (F1*ip2(i))+(F2*ip2(i))*(np.reshape(summation,-1)*ip2(i))

F = f.transpose()

start_time = time.time()

LHS = D @ L
RHS = F @ np.linalg.inv(P)

X = spsolve(LHS,RHS)

solve_time = time.time() - start_time

X = X[:,0]
mean_u = np.reshape(X,(n,n))


#Define x and y as arrays between -1 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y) 
err_inf, err_sq = compute_err(mean_u, U_exact)
print('Total time:', solve_time, '\nErr_inf:', err_inf, '\nErr sq:', err_sq)
